In [1]:
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from imblearn.over_sampling import RandomOverSampler
from sklearn.ensemble import IsolationForest
import seaborn as sns
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd 

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))



/kaggle/input/mobile-price-classification/train.csv
/kaggle/input/mobile-price-classification/test.csv


In [2]:
train = pd.read_csv('/kaggle/input/mobile-price-classification/train.csv')
test = pd.read_csv('/kaggle/input/mobile-price-classification/test.csv')

In [3]:
test.set_index('id')

,battery_power,blue,clock_speed,dual_sim,fc,four_g,int_memory,m_dep,mobile_wt,n_cores,pc,px_height,px_width,ram,sc_h,sc_w,talk_time,three_g,touch_screen,wifi
id,,,,,,,,,,,,,,,,,,,,
1,1043,1,1.8,1,14,0,5,0.1,193,3,16,226,1412,3476,12,7,2,0,1,0
2,841,1,0.5,1,4,1,61,0.8,191,5,12,746,857,3895,6,0,7,1,0,0
3,1807,1,2.8,0,1,0,27,0.9,186,3,4,1270,1366,2396,17,10,10,0,1,1
4,1546,0,0.5,1,18,1,25,0.5,96,8,20,295,1752,3893,10,0,7,1,1,0
5,1434,0,1.4,0,11,1,49,0.5,108,6,18,749,810,1773,15,8,7,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
996,1700,1,1.9,0,0,1,54,0.5,170,7,17,644,913,2121,14,8,15,1,1,0
997,609,0,1.8,1,0,0,13,0.9,186,4,2,1152,1632,1933,8,1,19,0,1,1
998,1185,0,1.4,0,1,1,8,0.5,80,1,12,477,825,1223,5,0,14,1,0,0


Setting ID as index

In [ ]:
for i in train.columns:
    print(train[i].value_counts())

In [4]:
corr = train.corr()
corr.style.background_gradient(cmap = 'coolwarm')

,battery_power,blue,clock_speed,dual_sim,fc,four_g,int_memory,m_dep,mobile_wt,n_cores,pc,px_height,px_width,ram,sc_h,sc_w,talk_time,three_g,touch_screen,wifi,price_range
battery_power,1.000000,0.011252,0.011482,-0.041847,0.033334,0.015665,-0.004004,0.034085,0.001844,-0.029727,0.031441,0.014901,-0.008402,-0.000653,-0.029959,-0.021421,0.052510,0.011522,-0.010516,-0.008343,0.200723
blue,0.011252,1.000000,0.021419,0.035198,0.003593,0.013443,0.041177,0.004049,-0.008605,0.036161,-0.009952,-0.006872,-0.041533,0.026351,-0.002952,0.000613,0.013934,-0.030236,0.010061,-0.021863,0.020573
clock_speed,0.011482,0.021419,1.000000,-0.001315,-0.000434,-0.043073,0.006545,-0.014364,0.012350,-0.005724,-0.005245,-0.014523,-0.009476,0.003443,-0.029078,-0.007378,-0.011432,-0.046433,0.019756,-0.024471,-0.006606
dual_sim,-0.041847,0.035198,-0.001315,1.000000,-0.029123,0.003187,-0.015679,-0.022142,-0.008979,-0.024658,-0.017143,-0.020875,0.014291,0.041072,-0.011949,-0.016666,-0.039404,-0.014008,-0.017117,0.022740,0.017444
fc,0.033334,0.003593,-0.000434,-0.029123,1.000000,-0.016560,-0.029133,-0.001791,0.023618,-0.013356,0.644595,-0.009990,-0.005176,0.015099,-0.011014,-0.012373,-0.006829,0.001793,-0.014828,0.020085,0.021998
four_g,0.015665,0.013443,-0.043073,0.003187,-0.016560,1.000000,0.008690,-0.001823,-0.016537,-0.029706,-0.005598,-0.019236,0.007448,0.007313,0.027166,0.037005,-0.046628,0.584246,0.016758,-0.017620,0.014772
int_memory,-0.004004,0.041177,0.006545,-0.015679,-0.029133,0.008690,1.000000,0.006886,-0.034214,-0.028310,-0.033273,0.010441,-0.008335,0.032813,0.037771,0.011731,-0.002790,-0.009366,-0.026999,0.006993,0.044435
m_dep,0.034085,0.004049,-0.014364,-0.022142,-0.001791,-0.001823,0.006886,1.000000,0.021756,-0.003504,0.026282,0.025263,0.023566,-0.009434,-0.025348,-0.018388,0.017003,-0.012065,-0.002638,-0.028353,0.000853
mobile_wt,0.001844,-0.008605,0.012350,-0.008979,0.023618,-0.016537,-0.034214,0.021756,1.000000,-0.018989,0.018844,0.000939,0.000090,-0.002581,-0.033855,-0.020761,0.006209,0.001551,-0.014368,-0.000409,-0.030302
n_cores,-0.029727,0.036161,-0.005724,-0.024658,-0.013356,-0.029706,-0.028310,-0.003504,-0.018989,1.000000,-0.001193,-0.006872,0.024480,0.004868,-0.000315,0.025826,0.013148,-0.014733,0.023774,-0.009964,0.004399


Checking for correlation among features and target variable


In [ ]:
train.isna().sum()
train.duplicated().sum()



Checking for duplicated rows


In [ ]:
m1 = IsolationForest()
m1.fit(train)
train['anamoly'] = m1.predict(train)  
train.drop(train[train.anamoly == -1], inplace = True)

Isolation Forest is used for detecting outliers.
Isolation forest not used here beacuse  1700 were outliers outta 2000

In [5]:
y = train.price_range
train.drop(columns = ['price_range'], inplace = True)
trainx, testx, trainy, testy =  train_test_split(train, y, random_state = 100, test_size = 0.35 )

Splitting the train set into train and test


In [ ]:
sns.countplot(y)


Checking to see whether the target variable is balanced or not. In this case it is balanced. Else randomoversampler could have been used to balance the target variable.

In [7]:
model = LogisticRegression(max_iter = 1000000)
model.fit(trainx, trainy)
model.score(testx, testy)

/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of f AND g EVALUATIONS EXCEEDS LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


0.7728571428571429

Used LogisticRegression
The accuracy was 77%

In [ ]:
model = DecisionTreeClassifier()
min_split = np.array([2, 3, 4, 5, 6, 7, 8])
max_d = np.array(range(1,15))
alg = ['entropy', 'gini']
values_grid = {'min_samples_split': min_split, 'max_depth' : max_d, 'criterion' : alg}
rdecisiontree = RandomizedSearchCV(model, values_grid, cv = 5)
rdecisiontree.fit(trainx, trainy)
print('min_samples_split', rdecisiontree.best_estimator_.min_samples_split)
print('max_depth', rdecisiontree.best_estimator_.max_depth)
print('criterion', rdecisiontree.best_estimator_.criterion)
print('score', rdecisiontree.best_score_)

Used RandomizedsearchCV for parameter tuning and used the values with the best score


In [8]:
model = DecisionTreeClassifier(min_samples_split = 7, max_depth= 13, criterion = 'entropy', random_state = 100)
model.fit(trainx, trainy)
model.score(testx, testy)

0.8442857142857143

Used the values from RandomizedSearchCV and the results were predicted with accuracy of 84%  

In [ ]:
model = RandomForestClassifier()
min_split = np.array(range(2, 10))
max_d = np.array(range(1, 15))
alg = ['gini', 'entropy']
nest = np.array(range(1, 100))
valuesgrid = {'min_samples_split': min_split, 'max_depth': max_d, 'criterion': alg, 'n_estimators': nest}
rrandomforest = RandomizedSearchCV(model, valuesgrid, cv = 5)
rrandomforest.fit(trainx, trainy)
print('min_samples_split', rrandomforest.best_estimator_.min_samples_split)
print('max_depth', rrandomforest.best_estimator_.max_depth)
print('criterion', rrandomforest.best_estimator_.criterion)
print('n_estimators', rrandomforest.best_estimator_.n_estimators)
print('score', rrandomforest.best_score_)

Used RandomizedsearchCV for parameter tuning and used the values with the best score

In [9]:
model = RandomForestClassifier(min_samples_split = 9, max_depth = 13, criterion = 'entropy', n_estimators = 76, random_state = 100)
model.fit(trainx, trainy)
model.score(testx, testy)

0.8671428571428571

Improved prediction with an accuracy of 87%

In [10]:
model = GaussianNB()
model.fit(trainx, trainy)
model.score(testx, testy)

0.8071428571428572

Used GaussianNB which predicted with an accuracy of 81%


In [ ]:
model = SVC()
parameters = {'C': [0.001, 0.01, 0.1, 1, 10, 100], 'gamma': [0.001, 0.01, 0.1, 1, 10, 100],'kernel': ['rbf', 'poly', 'linear', 'sigmoid’, ‘precomputed'] }
rsvc = RandomizedSearchCV(model, parameters, cv = 5)
rsvc.fit(trainx, trainy)
print('best c', rsvc.best_estimator_.C)
print('best gamma', rsvc.best_estimator_.gamma)
print('best kernel', rsvc.best_estimator_.kernel)
print('best score', rsvc.best_score_)

Used RandomizedsearchCV for parameter tuning and used the values with the best score

In [11]:
model = SVC(C = 100 , gamma = 100 , kernel = 'linear', random_state = 100)
model.fit(trainx, trainy)
model.score(testx, testy)

0.9628571428571429

SVC gave the best prediction with an accuracy of 96%. This is the best model
